In [1]:
import cv2
import json
import numpy as np
import random
from copy import deepcopy

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

# set seed

In [2]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

g = torch.Generator()
g.manual_seed(0)

# dataset

In [3]:
class CustomDataset(Dataset):
    def __init__(self, json_path_list):
        self.annotations = []
        for json_path in json_path_list:
            with open(json_path, 'r') as f:
                annotations = json.load(f)
            self.annotations.extend(annotations['annotations'])
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        anno = self.annotations[idx]
        label = anno['label']
        label = np.eye(5)[label].astype(np.float32)
        
        # label smoothing
#         label = label * .9 + 0.1 / 5 
        
        kpts_ = np.array(anno['kpts']).reshape(-1, 2)
        kpts_ = preprocessing(kpts_)
        
        return label, kpts_.astype(np.float32)
    
    
def preprocessing(kpts):
    kpts_ = deepcopy(kpts)
    kpts_ = (kpts_ - kpts_.min(axis=0)) / (kpts_.max(axis=0) - kpts_.min(axis=0))
    return kpts_.flatten()

In [4]:
ds = CustomDataset([f'./data/annotation_{i}.json' for i in range(5)])
len(ds)

2513

In [5]:
ds[0]

(array([1., 0., 0., 0., 0.], dtype=float32),
 array([0.21594635, 1.        , 0.443099  , 0.9287557 , 0.6469579 ,
        0.7714337 , 0.8361737 , 0.6478518 , 1.        , 0.57608795,
        0.49768278, 0.4669195 , 0.509404  , 0.2612543 , 0.503457  ,
        0.12828362, 0.49311274, 0.        , 0.34171033, 0.47398508,
        0.27542248, 0.46475977, 0.304567  , 0.6385949 , 0.35247087,
        0.6362982 , 0.19208448, 0.5076398 , 0.12309471, 0.5348637 ,
        0.18273547, 0.69105816, 0.24475259, 0.6733748 , 0.05024889,
        0.5485357 , 0.        , 0.564812  , 0.06049323, 0.6850807 ,
        0.11008564, 0.7059657 ], dtype=float32))

In [6]:
total = len(ds)
num_train = int(total * .8)
ds_train, ds_valid = torch.utils.data.random_split(ds, [num_train, total - num_train], generator=g)
print(f'train: {len(ds_train)} / valid: {len(ds_valid)}')

train: 2010 / valid: 503


In [7]:
labels, cnt = np.unique([ds_valid.dataset.annotations[i]['label'] for i in ds_valid.indices], return_counts=True)
print({l:c for l, c in zip(labels, cnt)})

{0: 110, 1: 98, 2: 100, 3: 100, 4: 95}


# model

In [8]:
m = nn.BatchNorm1d(100)
# Without Learnable Parameters
m = nn.BatchNorm1d(100, affine=False)
input = torch.randn(20, 100)
output = m(input)

In [9]:
output.shape

torch.Size([20, 100])

In [10]:
simple_model = nn.Sequential(
    nn.Linear(42, 20),
    nn.BatchNorm1d(20),
    nn.ReLU(True,),
    nn.Linear(20, 10),
    nn.BatchNorm1d(10),
    nn.ReLU(True,),
    nn.Linear(10, 5),
    nn.Sigmoid(),
)

In [11]:
test = simple_model(torch.randn(2, 42))
print('shape:', test.size())

shape: torch.Size([2, 5])


# training

In [13]:
EPOCHS = 40
BS = 64

In [14]:
train_loader = DataLoader(ds_train, BS, shuffle=True, drop_last=True, num_workers=8)
valid_loader = DataLoader(ds_valid, BS, shuffle=False, drop_last=False, num_workers=8)

In [15]:
# criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(simple_model.parameters(), lr=2e-3)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=2e-3, epochs=EPOCHS, steps_per_epoch=len(train_loader))

baseline: 0.872255489021956

label smoothing: 0.8343313373253493

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
simple_model = simple_model.to(device)

all_valid_pred = []
all_valid_label = []

for epoch in range(EPOCHS):
    simple_model.train()
    epoch_loss = 0.
    for label, input_ in tqdm(train_loader):
        label, input_ = label.to(device), input_.to(device)
        
        pred = simple_model(input_)
        loss = criterion(pred, label)
        
        epoch_loss += loss.detach().item() * label.size()[0]
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
    print(f'[epoch {epoch}] loss: {epoch_loss / len(ds_train)}, lr: {scheduler.get_lr()}')
    
    simple_model.eval()
    
    correct = 0
    for label, input_ in tqdm(valid_loader):
        label, input_ = label.to(device), input_.to(device)
        
        with torch.no_grad():
            pred = simple_model(input_)
            cls_pred = pred.argmax(dim=-1)
            cls_true = label.argmax(dim=-1)
            
            correct += (cls_pred == cls_true).sum().item()
            
            if epoch == EPOCHS - 1:
                all_valid_pred.extend(cls_pred.detach().cpu().numpy().tolist())
                all_valid_label.extend(cls_true.detach().cpu().numpy().tolist())
            
    print(f'--val acc: {correct / len(ds_valid)}')

100%|██████████| 31/31 [00:00<00:00, 61.00it/s]
/usr/lib/python3/dist-packages/torch/optim/lr_scheduler.py:1289: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


[epoch 0] loss: 0.7094271133195108, lr: [0.00011288677002285622]


100%|██████████| 8/8 [00:00<00:00, 55.51it/s]


--val acc: 0.24055666003976142


100%|██████████| 31/31 [00:00<00:00, 134.50it/s]


...

--val acc: 1.0


100%|██████████| 31/31 [00:00<00:00, 138.32it/s]


[epoch 39] loss: 0.016058651665549966, lr: [1.4549800742174329e-08]


100%|██████████| 8/8 [00:00<00:00, 54.81it/s]

--val acc: 1.0


In [17]:
from sklearn.metrics import confusion_matrix

In [18]:
confusion_matrix(all_valid_label, all_valid_pred)

array([[110,   0,   0,   0,   0],
       [  0,  98,   0,   0,   0],
       [  0,   0, 100,   0,   0],
       [  0,   0,   0, 100,   0],
       [  0,   0,   0,   0,  95]])

### base

acc: 0.9620758483033932


```python
array([[100,   4,   2,   0,   0],
       [  0, 102,   3,   0,   0],
       [  0,  10, 106,   0,   0],
       [  0,   0,   0,  84,   0],
       [  0,   0,   0,   0,  90]])
```

### label smoothing / 0.1

acc: 0.9481037924151696

```python
array([[101,   4,   1,   0,   0],
       [  1,  99,   5,   0,   0],
       [  0,  15, 101,   0,   0],
       [  0,   0,   0,  84,   0],
       [  0,   0,   0,   0,  90]])
```

### batchnorm

acc: 1.0

```python
array([[106,   0,   0,   0,   0],
       [  0, 105,   0,   0,   0],
       [  0,   0, 116,   0,   0],
       [  0,   0,   0,  84,   0],
       [  0,   0,   0,   0,  90]])
```

In [19]:
torch.onnx.export(simple_model, input_[:1], "simple_model.onnx", verbose=True, input_names=['input'], output_names=['output'])

graph(%input : Float(1, 42, strides=[42, 1], requires_grad=0, device=cuda:0),
      %0.weight : Float(20, 42, strides=[42, 1], requires_grad=1, device=cuda:0),
      %0.bias : Float(20, strides=[1], requires_grad=1, device=cuda:0),
      %1.weight : Float(20, strides=[1], requires_grad=1, device=cuda:0),
      %1.bias : Float(20, strides=[1], requires_grad=1, device=cuda:0),
      %1.running_mean : Float(20, strides=[1], requires_grad=0, device=cuda:0),
      %1.running_var : Float(20, strides=[1], requires_grad=0, device=cuda:0),
      %3.weight : Float(10, 20, strides=[20, 1], requires_grad=1, device=cuda:0),
      %3.bias : Float(10, strides=[1], requires_grad=1, device=cuda:0),
      %4.weight : Float(10, strides=[1], requires_grad=1, device=cuda:0),
      %4.bias : Float(10, strides=[1], requires_grad=1, device=cuda:0),
      %4.running_mean : Float(10, strides=[1], requires_grad=0, device=cuda:0),
      %4.running_var : Float(10, strides=[1], requires_grad=0, device=cuda:0),
    

In [20]:
import onnxruntime as ort
from tqdm import tqdm
import numpy as np

ort_session = ort.InferenceSession("simple_model.onnx")

for _ in tqdm(range(100)):
    outputs = ort_session.run(
        ['output'],
        {"input": np.random.randn(1, 42).astype(np.float32)},
    )
    

100%|██████████| 100/100 [00:00<00:00, 45669.69it/s]
